### 3. Scrape the list of third party drivers license locations from https://travel-id-documents.az.gov/authorized-third-party-driver-license-locations but include the link. Save as a CSV. Since it's more than just the text from the table, this requires actually using BeautifulSoup :(

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://travel-id-documents.az.gov/authorized-third-party-driver-license-locations'

# we're using some 'fuller' user-agent headers for this one, otherwise it doesn't really give us the table with our request.
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

response = requests.get(url)

print(response.status_code)
doc = BeautifulSoup(response.text, 'html.parser')
print(doc)

403
<!DOCTYPE html>
<html lang="en-US"><head><title>Just a moment...</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="noindex,nofollow" name="robots"/><meta content="width=device-width,initial-scale=1" name="viewport"/><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;max-width:60rem;padding-left:1.5rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{font-size:1.5rem;font-weight:500;line-height:2.25rem}@media (width <= 720px){.h2{font-size:1.25rem;line-height:1.5rem}}#challenge-error-text{background-image:url();background-repeat:no-repeat;bac

In [5]:
table = doc.find('table').find('tbody')

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
table_rows = table.find_all('tr')

In [4]:
# make sure to omit row 1 (zero) as its just the labels in the <thead> of the <table>
data = []
for row in table_rows[1:]:
  result = []
  if len(row.select('a')) > 0:
    result.append(row.select('a')[0]['href'])
  else:
    result.append('')
  cells = row.select('td')
  for cell in cells:
      result.append(cell.text)
  data.append(result)

NameError: name 'table_rows' is not defined

In [ ]:
df = []
df = pd.DataFrame(df, columns=["Link", "Company","Address","Telephone","Hours"]

In [ ]:
# Should see the first five rows of the table here
df.head()

In [ ]:
# Save to csv
df.to_csv("license-locations.csv")